In [5]:
# import libraries
import numpy as np
import sys

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col,countDistinct
from pyspark.sql.types import ArrayType, BooleanType, LongType, FloatType, IntegerType, DateType,StringType
from pyspark.sql import types as t

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify project") \
    .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
#Load dataset
#path = "mini_sparkify_event_data.json"
#sparkify_df = spark.read.json(path)
df = spark.read.json("mini_sparkify_event_data.json")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Path does not exist: file:/home/jupyter/jupyter_default_dir/mini_sparkify_event_data.json;'
Traceback (most recent call last):
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 274, in json
    return self._df(self._jreader.json(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: file:/home/jupyter/jupyter_default_dir/mini_sparkify_event_data.json;'



In [4]:
#Load dataset
path = "mini_sparkify_event_data.json"
sparkify_df = spark.read.json(path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'Path does not exist: file:/home/jupyter/jupyter_default_dir/mini_sparkify_event_data.json;'
Traceback (most recent call last):
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 274, in json
    return self._df(self._jreader.json(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/spark-2.4.3-bin-spark-2.4.3-bin-hadoop2.8/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Path does not exist: file:/home/jupyter/jupyter_default_dir/mini_sparkify_event_data.json;'



In [5]:
#Cast userID to integer datatype
sparkify_df = sparkify_df.withColumn("userId", sparkify_df ["userId"].cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [6]:
#Drop records without userids or sessionids
sparkify_df = sparkify_df.dropna(how = "any", subset = ["userId", "sessionId"])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [7]:
#Drop culumns not needed for now
sparkify_df = sparkify_df.drop(*["artist","song","firstName", "lastName"]) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [8]:
#Number of rows after doing some transformations
sparkify_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [9]:
#For a seleced set of columns, lets explore the unique value counts
columns=["auth",
"gender",
"level",
"location",
"method",
"page",
"userAgent",
"ts"]

for c in columns:
    sparkify_df.groupBy(c).count().orderBy("count").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [10]:
# sparkify_df = sparkify_df.withColumn("ts_str",col("ts").cast(StringType()))
# sparkify_df = sparkify_df.withColumn("reg_str",col("registration").cast(StringType()))
# sparkify_df=sparkify_df.withColumn("ts_time",F.to_timestamp(sparkify_df.ts_str))
# sparkify_df=sparkify_df.withColumn("registration_time",F.to_timestamp(sparkify_df.reg_str))
# sparkify_df.printSchema()


# representing registration and timstamp by creating new columns
# tss = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%m-%d-%Y %H:%M:%S"))
# sparkify_df = sparkify_df.withColumn('updated_ts', tss('ts'))
# downgrade_value  = udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
# sparkify_df = sparkify_df.withColumn("downgrade_value", downgrade_value("page"))

# Converting timestamps to datetime from epoch time, to get the hour of the day
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
# Adding a new column called hour to our dataframe
sparkify_df = sparkify_df.withColumn("hour", get_hour(sparkify_df.ts))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined



In [11]:
sparkify_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
name 'sparkify_df' is not defined
Traceback (most recent call last):
NameError: name 'sparkify_df' is not defined

